In [ ]:
! pip install mlflow

In [ ]:
! pip install xgboost

In [ ]:
import re
import mlrun
import mlflow

# MLflow Tracker test notebook

### First let's create the project and the context folder

In [ ]:
project_name1 = "test-mlflow-tracking1"
# we choose the first run option from above
mlrun.mlconf.external_platform_tracking.mlflow.match_experiment_to_runtime = True

# Create a project for this demo:
project = mlrun.get_or_create_project(name=project_name1, context="./test_mlflow_tracking")
project.set_model_monitoring_credentials(endpoint_store_connection="v3io",stream_path="v3io", tsdb_connection="v3io")

## Xgboost example function

In [ ]:
%%writefile  ./test_mlflow_tracking/training.py

import mlflow
import mlflow.xgboost
import xgboost as xgb
from mlflow import log_metric
from sklearn import datasets
from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import train_test_split

def example_xgb_run():
    # prepare train and test data
    iris = datasets.load_iris()
    X = iris.data
    y = iris.target
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    # enable auto logging
    mlflow.xgboost.autolog()

    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)

    with mlflow.start_run():
        # train model
        params = {
            "objective": "multi:softprob",
            "num_class": 3,
            "learning_rate": 0.3,
            "eval_metric": "mlogloss",
            "colsample_bytree": 1.0,
            "subsample": 1.0,
            "seed": 42,
        }
        model = xgb.train(params, dtrain, evals=[(dtrain, "train")])
        
        # evaluate model
        y_proba = model.predict(dtest)
        y_pred = y_proba.argmax(axis=1)
        loss = log_loss(y_test, y_proba)
        acc = accuracy_score(y_test, y_pred)

## Mlrun code 

### Need to change the mlrun config in order to use the tracker


In [ ]:

mlrun.mlconf.external_platform_tracking.enabled = True

### 3 possible way to run tracking:
1. We can set: 'mlrun.mlconf.external_platform_tracking.mlflow.match_experiment_to_runtime' to True, this determines the run id and is the safest way
2. We can set the experiment name at: 'mlflow.environment_variables.MLFLOW_EXPERIMENT_NAME.set', this determines the experiment and we track the run added to it
3. We can just run it, then we will look across all experiments for added runs, this is not encouraged

### Then we create the functions

In [ ]:

function_name = "example-xgb-run"
handler_name = "example_xgb_run"

# Create a MLRun function using the example train file (all the functions must be located in it):
training_func = project.set_function(
    func="training.py",
    name=function_name,
    kind="job",
    image="mlrun/mlrun",
)

### Now we run the function, and after that we can look at the UI and see all metrics and parameters are logged in mlrun 

In [ ]:
# we run the example code using mlrun
train_run = training_func.run(
    local=True, handler=handler_name,
)

## Now let's check that the artifacts were correctly created in the project

In [ ]:
def remove_iter_from_uri(uri):
    return re.sub(r"#\d+:latest", '', uri)

In [ ]:

handler_name = handler_name.replace("_", "-")
artifact_prefix = function_name + "-" + handler_name + "_"

feature_importance_weights_json = project.get_artifact(artifact_prefix + "feature_importance_weight_json", tag="latest")
feature_importance_weights_png = project.get_artifact(artifact_prefix + "feature_importance_weight_png", tag="latest")
model = project.list_models(name=artifact_prefix+'model', tag="latest", best_iteration=True)[0]

# remove the iter number from the artifact_uri
feature_importance_weights_json_uri = remove_iter_from_uri(feature_importance_weights_json.uri)
feature_importance_weights_png_uri= remove_iter_from_uri(feature_importance_weights_png.uri)
model_uri = remove_iter_from_uri(model.uri)

assert feature_importance_weights_json_uri
assert feature_importance_weights_png_uri
assert model_uri
assert feature_importance_weights_json_uri==train_run.outputs["feature_importance_weight_json"]
assert feature_importance_weights_png_uri==train_run.outputs["feature_importance_weight_png"]
# assert model_uri==train_run.outputs["model"]


# Now we will test this as a model server

### To use this as an model server we need to implement two functions, load and predict

In [ ]:
%%writefile ./test_mlflow_tracking/serving.py

import zipfile
from typing import Any, Dict, List, Union

import mlflow
import numpy as np
import os
import mlrun
from mlrun.serving.v2_serving import V2ModelServer
import xgboost as xgb
import pandas as pd

class MLFlowModelServer(V2ModelServer):
    """
    MLFlow tracker Model serving class, inheriting the V2ModelServer class for being initialized automatically by the model
    server and be able to run locally as part of a nuclio serverless function, or as part of a real-time pipeline.
    """

    def load(self):
        """
        loads an model that was logged by the MLFlow tracker model
        """
        # all we need to do is unzip the model dir and then use mlflow's load function
        model_file, _ = self.get_model(".zip")
        model_path_unzip = model_file.replace(".zip", "")

        with zipfile.ZipFile(model_file, "r") as zip_ref:
            zip_ref.extractall(model_path_unzip)
            
        self.model = mlflow.pyfunc.load_model(model_path_unzip)

    def predict(self, request: Dict[str, Any]) -> list:
        """
        Infer the inputs through the model. The inferred data will
        be read from the "inputs" key of the request.

        :param request: The request to the model using xgboost's predict. 
                The input to the model will be read from the "inputs" key.

        :return: The model's prediction on the given input.
        """
        
        # Get the inputs and set to accepted type:
        inputs = pd.DataFrame(request["inputs"])

        # Predict using the model's predict function:
        predictions = self.model.predict(inputs)

        # Return as list:
        return predictions.tolist()


### creating the server and serving function

In [ ]:
function_name = "example-xgb-server"
serving_func = project.set_function(
    func="serving.py",
    name="example-xgb-server",
    kind="serving",
    image="mlrun/mlrun",
    requirements=["xgboost"]
)

In [ ]:
# add the model
serving_func.add_model("mlflow_xgb_model", class_name="MLFlowModelServer", model_path=train_run.outputs["model"])

### Let's try to test our model 

In [ ]:
serving_func.deploy()

## Now let's check that the serving function was correctly created in the project

In [ ]:
func = project.get_function(function_name)
assert func
assert func==serving_func

# Offline tests

in this example we will run a function that's being logged by mlflow without mlrun,
and then import it into mlrun afterwards.

In [ ]:
%%writefile ./test_mlflow_tracking/offline_training.py

import mlflow
import mlflow.xgboost
import xgboost as xgb
from mlflow import log_metric
from sklearn import datasets
from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import train_test_split

mlflow.environment_variables.MLFLOW_EXPERIMENT_NAME.set("example_xgb_run")

# the function we run that is being logged by mlflow
def example_xgb_run():
    # prepare train and test data
    iris = datasets.load_iris()
    X = iris.data
    y = iris.target
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    # enable auto logging
    mlflow.xgboost.autolog()

    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)

    with mlflow.start_run(run_name="offline-import-run"):
        # train model
        params = {
            "objective": "multi:softprob",
            "num_class": 3,
            "learning_rate": 0.3,
            "eval_metric": "mlogloss",
            "colsample_bytree": 1.0,
            "subsample": 1.0,
            "seed": 42,
        }
        model = xgb.train(params, dtrain, evals=[(dtrain, "train")])
        
        # evaluate model
        y_proba = model.predict(dtest)
        y_pred = y_proba.argmax(axis=1)
        loss = log_loss(y_test, y_proba)
        acc = accuracy_score(y_test, y_pred)
        
        # log metrics by hand
        mlflow.log_metrics({"log_loss": loss, "accuracy": acc})

In [ ]:
import sys
import os
sys.path.insert(0, os.path.abspath("./"))
from test_mlflow_tracking.offline_training import example_xgb_run
import mlrun
from mlrun.track.trackers.mlflow_tracker import MLFlowTracker
import tempfile
import mlflow
# Allow all tracking
mlrun.mlconf.external_platform_tracking.enabled = True

## Import offline run

In [ ]:
project_name2 = "test-mlflow-tracking2"

# Create a project for this demo:
project = mlrun.get_or_create_project(name=project_name2, context="./test_mlflow_tracking")

# Create a MLRun function that we will log in to:
function_name = "example-xgb-run-offline"

training_func = project.set_function(
    func="offline_training.py",
    name=function_name,
    kind="job",
    image="mlrun/mlrun",
)

In [ ]:
# We create a temporary working area to avoid junk in ours    
with tempfile.TemporaryDirectory() as test_directory:
    mlflow.set_tracking_uri(test_directory)  # Tell mlflow where to save logged data

    # Run mlflow wrapped code
    example_xgb_run()

    # Set mlconf path to artifacts
    mlrun.mlconf.artifact_path = test_directory + "/artifact"
    
    # Find last ran mlflow run
    mlflow_run = mlflow.last_active_run()
    
    # Import the run into mlrun using the function we created earlier
    imported_run = MLFlowTracker().import_run(
        project=project,
        reference_id=mlflow_run.info.run_id,
        function_name=function_name,
    )



## Checking the run has been registred in the project and the artifacts created
Cannot check more than that since the runs/artifact names is auto-generated by mlflow and we currently don't support setting a name when importing an offline run

In [ ]:
runs = project.list_runs()
assert runs
assert len(runs)==1

artifacts = project.list_artifacts()
assert artifacts
assert len(artifacts)==3
assert artifacts[0]['metadata']['key']=='feature_importance_weight_json'
assert artifacts[1]['metadata']['key']=='feature_importance_weight_png'
assert artifacts[2]['metadata']['key']=='model'

models = project.list_models()
assert models
assert len(models)==1

## Import offline model

In [ ]:
project_name3 = "test-mlflow-tracking3"

# We create a temporary working area to avoid junk in ours    
with tempfile.TemporaryDirectory() as test_directory:
    mlflow.set_tracking_uri(test_directory)  # Tell mlflow where to save logged data

    # Run mlflow wrapped code
    example_xgb_run()
    
    # Create a project for this tester:
    project = mlrun.get_or_create_project(name=project_name3, context=test_directory)

    # Access model's uri through mlflow's last run
    mlflow_run = mlflow.last_active_run()
    model_uri = mlflow_run.info.artifact_uri + "/model"

    key = "test_model"
    MLFlowTracker().import_model(
        project=project,
        reference_id=model_uri,
        key=key,
        metrics=mlflow_run.data.metrics,
    )

    

## Checking if the model was logged into project

In [ ]:
# Validate model was logged into project
assert project.get_artifact(key)

## Now we test all the different mlflow logging options

In [ ]:
%%writefile ./test_mlflow_tracking/log_stuff.py

import json
import os
import mlflow
import matplotlib.pyplot as plt
from plotly import graph_objects as go
import numpy as np
from PIL import Image
import pandas as pd

mlflow.environment_variables.MLFLOW_EXPERIMENT_NAME.set("log_stuff")


def log_stuff():
    
    print("hey it's amit")
# ---------------------------------------------option 1------------------------------------------------
    # txt file to log
    features = "feature1, feature2, feature3, feature4, feature5"
    with open("features.txt", "w") as f:
        f.write(features)
        
# ---------------------------------------------option 2------------------------------------------------    
    # directory with txt/json files to log
    features = "dir_feature1, dir_feature2, dir_feature3, dir_feature4, dir_feature5"
    data = {"json_feature1": "val1", "json_feature2": 22222, "json_feature3": True}

    # Create couple of artifact files under the directory "data"
    os.makedirs("data", exist_ok=True)
    with open("data.json", "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2)
    with open("features.txt", "w") as f:
        f.write(features)
        
# ---------------------------------------------option 3------------------------------------------------
    
    matplotlib_fig, ax = plt.subplots()
    ax.plot([0, 1], [2, 3])
    
    plotly_fig = go.Figure(go.Scatter(x=[0, 1], y=[2, 3]))
    
# ---------------------------------------------option 4------------------------------------------------
    
    np_image = np.random.randint(0, 256, size=(100, 100, 3), dtype=np.uint8)
    PIL_image = Image.new("RGB", (100, 100))
    
# ---------------------------------------------option 5------------------------------------------------
    
    array = np.asarray([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
    np_dataset = mlflow.data.from_numpy(array)
    
    
    data = {
        'Name': ['John', 'Alice', 'Bob', 'Charlie'],
        'Age': [25, 28, 22, 30],
        'City': ['New York', 'San Francisco', 'Seattle', 'Boston']
    }
    df = pd.DataFrame(data)
    pd_dataset = mlflow.data.from_pandas(df)
    
# ---------------------------------------------option 6------------------------------------------------
    table_dict = {
        "inputs": ["What is MLflow?", "What is Databricks?"],
        "outputs": ["MLflow is ...", "Databricks is ..."],
        "toxicity": [0.0, 0.0],
    }
    
    
    
        
    with mlflow.start_run():
# ---------------------------------------------log option 1------------------------------------------------
        # log a txt file as an artifact
        mlflow.log_artifact("features.txt")
        
# ---------------------------------------------log option 2------------------------------------------------
        # log the files in a directory
        mlflow.log_artifacts("data", artifact_path="states")
        
# ---------------------------------------------log option 3------------------------------------------------
        # log matplotlib/plotly figures
        mlflow.log_figure(matplotlib_fig, "matplotlib_figure.png")
        mlflow.log_figure(plotly_fig, "plotly_figure.html")
        
# ---------------------------------------------log option 4------------------------------------------------
        # log np/PIL images
        mlflow.log_image(np_image, "np_image.png")
        mlflow.log_image(PIL_image, "PIL_image.png")
        
# ---------------------------------------------log option 5------------------------------------------------
        # log numpy/pandas dataset
        mlflow.log_input(np_dataset, context="numpy_training")
        mlflow.log_input(pd_dataset, context="pandas_training")
    
# ---------------------------------------------log option 6------------------------------------------------    
        # log a dict/dataframe as a table artifact (json format)
        mlflow.log_table(data=table_dict, artifact_file="dict_table.json")
        mlflow.log_table(data=df, artifact_file="df_table.json")
    
    
    
        # log string 
        mlflow.log_text("text1", "file1.txt")
        
        # log dicts as json/yaml
        mlflow.log_dict({"json_key1":"json_val1"}, "data.json")
        mlflow.log_dict({"yaml_key1":"yaml_val1"}, "data.yaml")
        
        # log a numeric value
        mlflow.log_metric(key="metric_key", value=12345)
        mlflow.log_metrics({"metric_key1": 1, "metric_key2": 2})
        
        # log a string parameter
        mlflow.log_param(key="param_key", value="param_val")
        mlflow.log_params({"param_key1":"param_val1", "param_key2":"param_val2"})
        

In [ ]:
mlrun.mlconf.external_platform_tracking.enabled = True

In [ ]:
project_name4 = "test-mlflow-tracking4"

# we choose the first run option from above
mlrun.mlconf.external_platform_tracking.mlflow.match_experiment_to_runtime = True
mlflow.set_tracking_uri("./mlruns")


project = mlrun.get_or_create_project(name=project_name4, context="./test_mlflow_tracking")
function_name = "log-stuff-func"
handler_name = "log_stuff"

# Create a MLRun function
loging_func = project.set_function(
    func="log_stuff.py",
    name=function_name,
    kind="job",
    image="mlrun/mlrun",
)

In [ ]:
# Run the function
log_run = loging_func.run(
    local=True, handler=handler_name,
)

## Checking the logged artifacts/results

In [ ]:
# Extract the artifacts and the runs from the project

artifacts = project.list_artifacts()
artifact_keys = [art['metadata']['key'] for art in artifacts]

runs = project.list_runs()
results = runs[0]['status']['results']

In [ ]:
handler_name = handler_name.replace("_", "-")
artifact_prefix = function_name + "-" + handler_name + "_"

assert artifacts
assert len(artifacts)==11

for key in artifact_keys:
    # Check artifacts from the project exist in the run outputs
    assert key in log_run.outputs.keys()
    
    # Now checks the values are the same (store uri)
    artifact_uri = project.get_artifact(key=artifact_prefix+key).uri
    artifact_uri = remove_iter_from_uri(artifact_uri)
    assert artifact_uri==log_run.outputs[key]
    
# Check the logged results (metrics)
for result in results.keys():
    assert result in log_run.outputs.keys()
    assert results[result]==log_run.outputs[result]


## Deleting projects and relevant resources

In [ ]:
run_db = mlrun.get_run_db()
run_db.delete_project(project_name1, mlrun.common.schemas.constants.DeletionStrategy.cascade)
run_db.delete_project(project_name2, mlrun.common.schemas.constants.DeletionStrategy.cascade)
run_db.delete_project(project_name3, mlrun.common.schemas.constants.DeletionStrategy.cascade)
run_db.delete_project(project_name4, mlrun.common.schemas.constants.DeletionStrategy.cascade)

In [ ]:
import os
import shutil

files_to_delete = ["data.json", "features.txt"]
for file in files_to_delete:
    try:
        os.remove(file)
    except Exception as e:
        print(f"Error deleting file: {e}")


folders = ["test_mlflow_tracking", "data", "mlruns"]

for folder_name in folders:
    try:
        shutil.rmtree(folder_name)
        print(f"Folder '{folder_name}' deleted successfully.")
    except Exception as e:
        print(f"Error deleting folder: {e}")
